In [4]:
# import dependencies
import os
import csv
from pandas import DataFrame
import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt
import glob

In [5]:
# display column names in a csv  
path = '../etl/parking_citations/data/citations_db_ready/citations_0000000001.csv'
headers_list = pd.read_csv(path, sep=",")
headers_list.head(2)

,address,latitude,longitude,meter_type,parking_policy,policy_complexity,rate_range,rate_type,meter_id,street_cleaning,park_limit
0,501 UNION DR,34.057219,-118.268751,Single-Space,8A-6P Mon-Sat,13,$1.00,FLAT,WU150,12P-3P Mon,2.0
1,12200 VENICE BL,34.004300,-118.432007,Single-Space,8A-8P Mon-Sat,13,$1.00,FLAT,MV161,NaN,2.0


# Graph  1  |    Citations per Year / Month

In [3]:
# loop over csv files and extract columns needed

path = '../etl/parking_citations/data/citations_db_ready/'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, sep=",", usecols=[4, 11])
    li.append(df)
new_df = pd.concat(li)
print(f'There are: {len(new_df)} records in dataframe')
new_df.head()

There are: 8076329 records in dataframe


,issue_date,ticket_number
0,2015-12-21,1104803000
1,2015-12-26,1104820732
2,2015-12-21,1108321723
3,2015-12-19,1109139006
4,2015-12-26,1109455266


In [4]:
# convert "issue_date" column to a date 
new_df['issue_date'] = pd.to_datetime(new_df['issue_date'], dayfirst=True)

In [5]:
# sort dataframe rows between two dates-----(2015 - 2018)
start_date = "2015-01-01"
end_date = "2018-12-31"

mask = (new_df['issue_date'] > start_date) & (new_df['issue_date'] <= end_date)
new_df.loc[mask]
citations_2015_2018 = new_df.loc[mask]

print(f'There are: {len(citations_2015_2018)} records as of {end_date}')
print("=============================================================")
print(f'{len(new_df)-len(citations_2015_2018)} records were disregarded')
citations_2015_2018.head()

There are: 7333288 records as of 2018-12-31
743041 records were disregarded


,issue_date,ticket_number
0,2015-12-21,1104803000
1,2015-12-26,1104820732
2,2015-12-21,1108321723
3,2015-12-19,1109139006
4,2015-12-26,1109455266


In [6]:
# put year, month, and a day into separate columns
citations_2015_2018['Year'] = citations_2015_2018['issue_date'].dt.year
citations_2015_2018['Month'] = citations_2015_2018['issue_date'].dt.month
# citations_2015_2018['Day'] = citations_2015_2018['issue_date'].dt.day
citations_2015_2018.drop(["issue_date"], axis = 1, inplace = True)

citations_2015_2018.head()

/Users/pavelaltukhov/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/pavelaltukhov/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/pavelaltukhov/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

,ticket_number,Year,Month
0,1104803000,2015,12
1,1104820732,2015,12
2,1108321723,2015,12
3,1109139006,2015,12
4,1109455266,2015,12


In [7]:
# count citations by year and month
df_top_frequency = citations_2015_2018.groupby(['Year', 'Month'])['ticket_number'].agg(
    {"counts": len}).sort_values("Year", ascending=False).reset_index()

df_top_frequency.head()

/Users/pavelaltukhov/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


,Year,Month,counts
0,2018,12,124543
1,2018,6,140859
2,2018,1,180258
3,2018,3,184661
4,2018,4,177678


In [8]:
df_top_frequency.to_csv("tableau_data/citations_2015_2018.csv", encoding = 'utf-8', index = False)

# Graph  2  |    Top 20 Parking Violations

In [9]:
# loop over csv files and extract columns needed

path = '../etl/parking_citations/data/citations_db_ready/'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, sep=",", usecols=[4, 11, 13])
    li.append(df)
new_df = pd.concat(li)
print(f'There are: {len(new_df)} records in dataframe')
new_df.head()

/Users/pavelaltukhov/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


There are: 8076329 records in dataframe


,issue_date,ticket_number,violation_description
0,2015-12-21,1104803000,WHITE CURB
1,2015-12-26,1104820732,17104h
2,2015-12-21,1108321723,RED ZONE
3,2015-12-19,1109139006,HANDICAP/NO DP ID
4,2015-12-26,1109455266,PK IN PROH AREA


In [10]:
# convert "issue_date" column to a date 
new_df['issue_date'] = pd.to_datetime(new_df['issue_date'], dayfirst=True)

In [11]:
# sort dataframe rows between two dates-----(2015 - 2018)
start_date = "2015-01-01"
end_date = "2018-12-31"

mask = (new_df['issue_date'] > start_date) & (new_df['issue_date'] <= end_date)
new_df.loc[mask]
violations_2015_2018 = new_df.loc[mask]

print(f'There are: {len(violations_2015_2018)} records as of {end_date}')
print("=============================================================")
print(f'{len(new_df)-len(violations_2015_2018)} records were disregarded')
violations_2015_2018.head()

There are: 7333288 records as of 2018-12-31
743041 records were disregarded


,issue_date,ticket_number,violation_description
0,2015-12-21,1104803000,WHITE CURB
1,2015-12-26,1104820732,17104h
2,2015-12-21,1108321723,RED ZONE
3,2015-12-19,1109139006,HANDICAP/NO DP ID
4,2015-12-26,1109455266,PK IN PROH AREA


In [12]:
# put year, month, and a day into separate columns
violations_2015_2018['Year'] = violations_2015_2018['issue_date'].dt.year
violations_2015_2018.drop(["issue_date"], axis = 1, inplace = True)

violations_2015_2018.head()

/Users/pavelaltukhov/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/pavelaltukhov/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,ticket_number,violation_description,Year
0,1104803000,WHITE CURB,2015
1,1104820732,17104h,2015
2,1108321723,RED ZONE,2015
3,1109139006,HANDICAP/NO DP ID,2015
4,1109455266,PK IN PROH AREA,2015


In [13]:
# count citations by year and month
df_top_frequency = violations_2015_2018.groupby(['violation_description', 'Year'])['ticket_number'].agg(
    {"counts": len}).sort_values("counts", ascending=False).reset_index()

df_top_frequency

/Users/pavelaltukhov/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


,violation_description,Year,counts
0,NO PARK/STREET CLEAN,2017,628166
1,NO PARK/STREET CLEAN,2018,528835
2,NO PARK/STREET CLEAN,2016,518502
3,METER EXP.,2015,405365
4,METER EXP.,2016,388381
5,METER EXP.,2017,367144
6,NO PARK/STREET CLEAN,2015,356459
7,METER EXP.,2018,285800
8,RED ZONE,2017,169446
9,PREFERENTIAL PARKING,2017,147210


In [14]:
df_top_frequency.to_csv("tableau_data/violations_2015_2018.csv", encoding = 'utf-8', index = False)

# Graph  3  |    Parking Tickets Revenue

In [15]:
# loop over csv files and extract columns needed

path = '../etl/parking_citations/data/citations_db_ready/'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, sep=",", usecols=[3, 4, 5, 7, 11, 13])
    li.append(df)
new_df = pd.concat(li)
print(f'There are: {len(new_df)} records in dataframe')
new_df.head()

/Users/pavelaltukhov/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


There are: 8076329 records in dataframe


,fine_amount,issue_date,latitude,longitude,ticket_number,violation_description
0,58.0,2015-12-21,33.945212,-118.401151,1104803000,WHITE CURB
1,NaN,2015-12-26,33.945212,-118.401009,1104820732,17104h
2,93.0,2015-12-21,34.185788,-118.363082,1108321723,RED ZONE
3,363.0,2015-12-19,34.157607,-118.438092,1109139006,HANDICAP/NO DP ID
4,68.0,2015-12-26,34.126932,-118.311225,1109455266,PK IN PROH AREA


In [17]:
# convert "issue_date" column to a date 
new_df['issue_date'] = pd.to_datetime(new_df['issue_date'], dayfirst=True)

In [19]:
# sort dataframe rows between two dates-----(2015 - 2018)
start_date = "2015-01-01"
end_date = "2018-12-31"

mask = (new_df['issue_date'] > start_date) & (new_df['issue_date'] <= end_date)
new_df.loc[mask]
revenue_2015_2018 = new_df.loc[mask]

print(f'There are: {len(revenue_2015_2018)} records as of {end_date}')
print("=============================================================")
print(f'{len(new_df)-len(revenue_2015_2018)} records were disregarded')
revenue_2015_2018.head()

There are: 7333288 records as of 2018-12-31
743041 records were disregarded


,fine_amount,issue_date,latitude,longitude,ticket_number,violation_description
0,58.0,2015-12-21,33.945212,-118.401151,1104803000,WHITE CURB
1,NaN,2015-12-26,33.945212,-118.401009,1104820732,17104h
2,93.0,2015-12-21,34.185788,-118.363082,1108321723,RED ZONE
3,363.0,2015-12-19,34.157607,-118.438092,1109139006,HANDICAP/NO DP ID
4,68.0,2015-12-26,34.126932,-118.311225,1109455266,PK IN PROH AREA


In [23]:
# preview the missing values and the % of missing values in each column
df = revenue_2015_2018_V2
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

missing_values_table(df)

Selected dataframe has 6 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [21]:
revenue_2015_2018_V2 = revenue_2015_2018.dropna(axis = 0, how ='any')

In [22]:
print("Old data frame length:", len(revenue_2015_2018), "\nNew data frame length:",  
       len(revenue_2015_2018_V2), "\nNumber of rows with at least 1 NA value: ", 
       (len(revenue_2015_2018)-len(revenue_2015_2018_V2))) 

Old data frame length: 7333288 
New data frame length: 7330503 
Number of rows with at least 1 NA value:  2785


In [24]:
# put year, month, and a day into separate columns
revenue_2015_2018_V2['Year'] = revenue_2015_2018_V2['issue_date'].dt.year
revenue_2015_2018_V2['Month'] = revenue_2015_2018_V2['issue_date'].dt.month
# citations_2015_2018['Day'] = citations_2015_2018['issue_date'].dt.day
revenue_2015_2018_V2.drop(["issue_date"], axis = 1, inplace = True)

revenue_2015_2018_V2.head()

/Users/pavelaltukhov/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/pavelaltukhov/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/pavelaltukhov/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

,fine_amount,latitude,longitude,ticket_number,violation_description,Year,Month
0,58.0,33.945212,-118.401151,1104803000,WHITE CURB,2015,12
2,93.0,34.185788,-118.363082,1108321723,RED ZONE,2015,12
3,363.0,34.157607,-118.438092,1109139006,HANDICAP/NO DP ID,2015,12
4,68.0,34.126932,-118.311225,1109455266,PK IN PROH AREA,2015,12
5,68.0,34.126932,-118.311225,1109455270,PK IN PROH AREA,2015,12


In [26]:
# count citations by year and month
df_top_frequency = revenue_2015_2018_V2.groupby(['Year', 'Month', 'violation_description', 'fine_amount'])['ticket_number'].agg(
    {"counts": len}).sort_values("counts", ascending=False).reset_index()

df_top_frequency

/Users/pavelaltukhov/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


,Year,Month,violation_description,fine_amount,counts
0,2017,8,NO PARK/STREET CLEAN,73.0,59072
1,2016,12,NO PARK/STREET CLEAN,73.0,58559
2,2017,6,NO PARK/STREET CLEAN,73.0,57864
3,2017,3,NO PARK/STREET CLEAN,73.0,57224
4,2016,8,NO PARK/STREET CLEAN,73.0,56877
5,2017,5,NO PARK/STREET CLEAN,73.0,56735
6,2017,7,NO PARK/STREET CLEAN,73.0,55787
7,2017,12,NO PARK/STREET CLEAN,73.0,55377
8,2018,1,NO PARK/STREET CLEAN,73.0,51109
9,2018,3,NO PARK/STREET CLEAN,73.0,50976


In [29]:
df_top_frequency['revenue'] = df_top_frequency.fine_amount * df_top_frequency.counts
df_top_frequency.head()

,Year,Month,violation_description,fine_amount,counts,revenue
0,2017,8,NO PARK/STREET CLEAN,73.0,59072,4312256.0
1,2016,12,NO PARK/STREET CLEAN,73.0,58559,4274807.0
2,2017,6,NO PARK/STREET CLEAN,73.0,57864,4224072.0
3,2017,3,NO PARK/STREET CLEAN,73.0,57224,4177352.0
4,2016,8,NO PARK/STREET CLEAN,73.0,56877,4152021.0


In [30]:
df_top_frequency.to_csv("tableau_data/revenue_2015_2018.csv", encoding = 'utf-8', index = False)